# VinDr evaluation

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from hydra import compose, initialize
from sklearn.metrics import roc_auc_score
sys.path.append("/vol/biomedic3/mb121/causal-contrastive")
from classification.classification_module import ClassificationModule
from data_handling.mammo import VinDrDataModule
from evaluation.helper_functions import run_inference
os.chdir("/vol/biomedic3/mb121/causal-contrastive/evaluation")

In [ ]:
# Mapping from human readable run name to Weights&Biases run_id. 

# Human readable name should be in format:
# for finetuning:
# {simclr/simclrcf/simclrcfaug}-{train_prop}-{seed}
# for linear probing
# {simclr/simclrcf/simclrcfaug}head-{train_prop}-{seed}

model_dict = {
    "supervised-0.1-33": "p6vg0bqi",
    "supervised-0.5-33": "cwi8xrg8",
    "supervised-1.0-33": "14q2njao",
    "supervised-0.1-22": "s8miws7d",
    "supervised-0.5-22": "x2muh4t0",
    "supervised-1.0-22": "mbdkezyb",
    "supervised-0.1-11": "exzxnswf",
    "supervised-0.5-11": "lvusaac3",
    "supervised-1.0-11": "tnzfk6l2",
    "supervised-0.05-11": "2ms8yvko",
    "supervised-0.05-22": "pobuespp",
    "supervised-0.05-33": "a81wveb4",
    "supervised-0.25-11": "thaje7u1",
    "supervised-0.25-22": "wq8yo3j8",
    "supervised-0.25-33": "pvs3glhw",
    "simclrcfhead-0.05-22": "hczqw8hi",
    "simclrhead-0.05-22": "39bvbdkz",
    "simclrcfhead-0.1-22": "bh8mayg5",
    "simclrhead-0.1-22": "yez5128j",
    "simclrcfhead-0.05-11": "wvl2zdo8",
    "simclrcfhead-0.1-11": "vly7y62m",
    "simclrhead-0.05-11": "ucuv8jkw",
    "simclrcfhead-0.05-33": "5roc5ir1",
    "simclrhead-0.1-11": "c7c41whj",
    "simclrcfhead-0.5-33": "y51jnbxy",
    "simclrhead-0.05-33": "gxsma4ff",
    "simclrcfhead-0.5-22": "vp8kutkn",
    "simclrhead-0.1-33": "npddj61e",
    "simclrcfhead-0.5-11": "ymgvfwto",
    "simclrhead-0.5-33": "ytd2saxv",
    "simclrcfhead-0.25-22": "7fz5b8qq",
    "simclrhead-0.5-22": "h8q836y7",
    "simclrhead-0.5-11": "ndqzz0vi",
    "simclrcfhead-0.25-11": "9wbih7s2",
    "simclrcfhead-1.0-11": "5b7zrnis",
    "simclrcfhead-0.25-33": "wjipoqjo",
    "simclrhead-0.25-22": "yajr0hg0",
    "simclrcfhead-1.0-33": "y92i0w3b",
    "simclrhead-1.0-22": "ugsl9v5g",
    "simclrcfhead-0.1-33": "h9e6d77k",
    "simclrcfhead-1.0-22": "egm73okb",
    "simclrhead-0.25-11": "by2uphj8",
    "simclrhead-1.0-11": "q0qc4dus",
    "simclrhead-0.25-33": "w0hnt490",
    "simclrhead-1.0-33": "v6q7usva",
    "simclrcfaughead-0.05-11": "kj0hbad5",
    "simclrcfaughead-0.1-11": "cr9kyz0t",
    "simclrcfaug-0.05-11": "rysd390e",
    "simclr-0.05-11": "9l7y5ut7",
    "simclrcfaug-0.1-11": "znaz1mde",
    "simclrcf-0.05-11": "p95njer4",
    "simclrcfaughead-0.25-11": "863y733o",
    "simclrcf-0.1-11": "iw7tlt92",
    "simclrcfaughead-0.5-11": "xzne0qa8",
    "simclrcfaug-1.0-11": "nyvzr6ob",
    "simclrcfaug-0.05-22": "fhcjybmj",
    "simclr-0.1-11": "mof52mfw",
    "simclrcfaug-0.1-22": "vg3xsy4o",
    "simclrcfaug-0.25-22": "hx1na3m8",
    "simclrcf-0.25-11": "ct68hj4b",
    "simclrcfaughead-1.0-11": "k67i29h3",
    "simclr-0.25-11": "2458sxig",
    "simclrcfaug-0.5-22": "31sfch4u",
    "simclrcf-0.5-11": "keifisvj",
    "simclrcfaug-1.0-22": "nldvcmd9",
    "simclrcfaughead-0.05-22": "t5gpmpkb",
    "simclrcfaughead-0.1-22": "zg79bbvy",
    "simclrcfaughead-0.25-22": "qaxfef1v",
    "simclr-0.5-11": "x4h890xg",
    "simclrcfaughead-0.5-22": "t731ufl6",
    "simclrcf-1.0-11": "snfp6z9e",
    "simclrcfaughead-1.0-22": "92ydgwzx",
    "simclr-1.0-11": "98a4nxz6",
    "simclrcf-0.05-22": "9shpau2m",
    "simclrcf-0.1-22": "7g2mzjji",
    "simclr-0.05-22": "bisabus9",
    "simclr-0.1-22": "0gczhnmx",
    "simclrcf-0.25-22": "av41b9aw",
    "simclr-0.25-22": "ehunbb4k",
    "simclrcf-0.5-22": "z28lpj3b",
    "simclr-0.5-22": "ic2srqqw",
    "simclrcf-1.0-22": "f3yuxs0n",
    "simclr-1.0-22": "n5w3srew",
    "simclrcf-0.05-33": "mvccg1m1",
    "simclr-0.05-33": "c73xfh8k",
    "simclrcf-0.1-33": "jgh63ufx",
    "simclrcf-0.25-33": "yzsrtgah",
    "simclrcf-0.5-33": "4r6q203i",
    "simclrcfaug-0.05-33": "byblnw9g",
    "simclrcfaug-0.1-33": "4ibb1j7n",
    "simclr-0.25-33": "cf5trd5v",
    "simclrcfaug-0.25-33": "b1qmartk",
    "simclrcfaug-0.5-33": "gkgfkfui",
    "simclrcf-1.0-33": "exxhffot",
    "simclr-0.1-33": "knuou61x",
    "simclrcfaug-1.0-33": "bozj3y17",
    "simclr-0.5-33": "qfbtg6f9",
    "simclrcfaughead-0.05-33": "ou1992k9",
    "simclr-1.0-33": "wh3lxhxz",
    "simclrcfaughead-0.1-33": "1zvalb0n",
    "simclrcfaughead-0.5-33": "u4wcx2c4",
    "simclrcfaughead-1.0-33": "opprmt0a",
    "simclrcfaughead-0.25-33": "vrf6l31v",
    "simclrcfaughead-0.5-33": "xquo6h56",
    "simclrcfaug-0.25-11": "ciucf48v",
    "simclrcfaug-0.5-11": "d98hdcn0",
}

In [ ]:
with initialize(version_base=None, config_path="../configs"):
    cfg = compose(
        config_name="config.yaml",
        overrides=["experiment=base_density", "data=vindr", "data.cache=False"],
    )
    data_module = VinDrDataModule(config=cfg)
test_loader = data_module.test_dataloader()

In [ ]:
df = pd.read_csv(f"../outputs/vindr2.csv")
for run_name, run_id in model_dict.items():
    already_in_df = run_name in df.run_name.values
    if run_id != "" and not already_in_df:
        print(run_name)
        model_to_evaluate = f"../outputs/run_{run_id}/best.ckpt"
        classification_model = ClassificationModule.load_from_checkpoint(
            model_to_evaluate, map_location="cuda"
        ).model.eval()
        classification_model.cuda()

        # ID evaluation
        inference_results = run_inference(test_loader, classification_model)
        scanners = inference_results["scanners"]

        for i in np.unique(scanners):
            sc_idx = np.where(scanners == i)
            targets = inference_results["targets"][sc_idx]
            preds = np.argmax(inference_results["confs"], 1)[sc_idx]
            confs = inference_results["confs"][sc_idx]
            if np.unique(targets).shape[0] == 4:
                res = {}
                res["N_test"] = [targets.shape[0]]
                res["ROC"] = [roc_auc_score(targets, confs, multi_class="ovr")]
                res["Model Name"] = [f"(OOD) VinDr\n{i}"]
                res["run_name"] = run_name
                print(res)
                df = pd.concat([df, pd.DataFrame(res, index=[0])], ignore_index=True)

        targets = inference_results["targets"]
        preds = np.argmax(inference_results["confs"], 1)
        confs = inference_results["confs"]
        res = {}
        res["N_test"] = [targets.shape[0]]
        res["ROC"] = [roc_auc_score(targets, confs, multi_class="ovr")]
        res["Model Name"] = [f"(OOD) VinDr"]
        res["run_name"] = run_name
        print(res)
        df = pd.concat([df, pd.DataFrame(res, index=[0])], ignore_index=True)
        df.to_csv(f"../outputs/vindr2.csv", index=False)